# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'KQ854.854.JT96.8'

# the auction goes:
auction = ["PAD_START", "PAD_START", "1C", "PASS", "1H", "PASS", "3S", "PASS", "4D", "PASS", "4H", "PASS", "4N", "PASS", "5H", "PASS", "6H", "PASS", "PASS", "PASS"]

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [4]:
lead.card

SK

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [5]:
lead.to_dict()['candidates']

[{'card': 'SK',
  'insta_score': 0.263,
  'expected_tricks_sd': 12.65,
  'p_make_contract': 0.94},
 {'card': 'DJ',
  'insta_score': 0.414,
  'expected_tricks_sd': 12.54,
  'p_make_contract': 0.91},
 {'card': 'C8',
  'insta_score': 0.232,
  'expected_tricks_sd': 12.55,
  'p_make_contract': 0.91}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [6]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['.A9xx.KQx.AKQJxx 9xxx.K.8xxxx.9xx AJTx.QJTxx.A.Txx 0.12598',
 'x.KJxx.Ax.AKQJxx Jxxx.9.Q8xx.9xxx AT9.AQTxx.Kxx.Tx 0.10436',
 'x.AK9x.Kxx.AKQTx JTxx.T.Q8xx.xxxx A9x.QJxxx.Ax.J9x 0.09034',
 'x.ATxx.AKx.AKQTx JT9x.x.8xx.J9xxx Axx.KQJ9x.Qxx.xx 0.08665',
 'J.AJxx.Kx.AKQTxx xxx.x.Q8xxx.J9xx AT9x.KQT9x.Ax.xx 0.08650',
 'A.AT9x.K8x.AKQxx xxx.Jx.Qxx.JT9xx JT9x.KQxx.Axx.xx 0.08297',
 'A.KJ9x.KQx.AK9xx Txxxx.xx.xxx.QTx J9.AQTx.A8x.Jxxx 0.07943',
 'x.AT9x.AK8.AKQTx T9xx.xx.xxxx.9xx AJx.KQJx.Qx.Jxxx 0.07930',
 '9.KJ9x.AK.AKQJxx Jxxx.x.Qxxxx.xxx ATx.AQTxx.8x.T9x 0.07679',
 'A.KJTx.AKx.KT9xx JTxxx.x.xxxx.Axx 9x.AQ9xx.Q8.QJxx 0.07632']